In [ ]:
import torch.nn.functional as F
import torch
import torch.nn as nn

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Implementation of Resnet18

Ref: https://github.com/aladdinpersson/Machine-Learning-Collection/tree/master/ML/Pytorch 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, kernel_size=3):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion*out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion*out_channels, kernel_size=kernel_size, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet18(nn.Module):
    def __init__(self, block, num_blocks=[2,2,2,2], num_classes=10, kernel_size=3, pool_size=4):
        super(ResNet18, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=kernel_size, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1, kernel_size=kernel_size)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2, kernel_size=kernel_size)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2, kernel_size=kernel_size)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2, kernel_size=kernel_size)
        self.avgpool = nn.AvgPool2d(pool_size)
        self.fc = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride, kernel_size):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride, kernel_size=kernel_size))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


The standard implementation of resnet18 consists of close to 11 million trainable parametersparameters.

In [ ]:
model = ResNet18(BasicBlock)
print(count_parameters(model))

12550218


Since our goal is to bring down the parameter count to less than 5 million as per the one of the goal of this mini project, we can play with layers, kernel size and then we can further try to train the model to get an accuracy that is higher than 90%

The initial configuration had blocks = [2,2,2,2], kernel_size = 3, pool_size = 4
We can try to reduce kernel size and see what is the result.
Let's go ahead with kernel_size = 2

In [ ]:
model = ResNet18(BasicBlock, [2,2,2,2],kernel_size=2)
print(count_parameters(model))

11689098


We can see that there is not much change in the parameters, let's try next by reducing the num_blocks from 2 to 1 in each 4.
num_blocks = [1,1,1,1]

In [ ]:
model = ResNet18(BasicBlock,  num_blocks = [1,1,1,1],kernel_size=2)
print(count_parameters(model))

5418378


We can see that there is some significant impact in the number of parameters, but the count is still above 5 million, let's again downgrade the pool_size from 4 to 2 pool_size = 2

In [ ]:
model = ResNet18(BasicBlock,  num_blocks = [1,1,1,1],kernel_size=2, pool_size=2)
print(count_parameters(model))

5418378


Let's increase the pool size and see what is the result...


In [ ]:
model = ResNet18(BasicBlock,  num_blocks = [1,1,1,1],kernel_size=2, pool_size=8)
print(count_parameters(model))

5418378


We can see that increasing pool size or decreasing pool size is not making the change in the number of parameters.

Let's see if reducing the kernel size from 0 to 1 works or not ?

In [ ]:
model = ResNet18(BasicBlock,  num_blocks = [1,1,1,1],kernel_size=1,pool_size=4)
print(count_parameters(model))

4901706


okay, so here we are and finally our num of parameters came down to 4901706 and we can finalize the same for further training and other manipulations.